# sklearn-porter

Repository: https://github.com/nok/sklearn-porter

## AdaBoostClassifier

Documentation: [sklearn.ensemble.AdaBoostClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html)

### Loading data:

In [1]:
from sklearn.datasets import load_iris

iris_data = load_iris()
X = iris_data.data
y = iris_data.target

print(X.shape, y.shape)

((150, 4), (150,))


### Train classifier:

In [2]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

base_estimator = DecisionTreeClassifier(max_depth=4, random_state=0)
clf = AdaBoostClassifier(base_estimator=base_estimator, n_estimators=100,
                         random_state=0)
clf.fit(X, y)

AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=4,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=0,
            splitter='best'),
          learning_rate=1.0, n_estimators=100, random_state=0)

### Transpile classifier:

In [4]:
%%time

from sklearn_porter import Porter

porter = Porter(clf, language='js')
output = porter.export(export_data=True)

with open('AdaBoostClassifier.js', 'w') as f:
    f.write(output)

CPU times: user 3.12 ms, sys: 2.27 ms, total: 5.39 ms
Wall time: 3.61 ms


Classifier:

In [5]:
%%bash

cat AdaBoostClassifier.js

if (typeof XMLHttpRequest === 'undefined') {
    var XMLHttpRequest = require("xmlhttprequest").XMLHttpRequest;
}

var AdaBoostClassifier = function(jsonFile) {
    this.forest = undefined;

    var promise = new Promise(function(resolve, reject) {
        var httpRequest = new XMLHttpRequest();
        httpRequest.onreadystatechange = function() {
            if (httpRequest.readyState === 4) {
                if (httpRequest.status === 200) {
                    resolve(JSON.parse(httpRequest.responseText));
                } else {
                    reject(new Error(httpRequest.statusText));
                }
            }
        };
        httpRequest.open('GET', jsonFile, true);
        httpRequest.send();
    });

    var imax = function(nums) {
        var index = 0;
        for (var i=0, l=nums.length; i < l; i++) {
            index = nums[i] > nums[index] ? i : index;
        }
        return index;
    };

    var predict = function(tree, features, node) {
        if (tre

Parameters:

In [6]:
%%bash

cat data.json | python -m json.tool

[
    {
        "childrenLeft": [
            1,
            -1,
            3,
            4,
            5,
            -1,
            -1,
            8,
            -1,
            -1,
            11,
            12,
            -1,
            -1,
            -1
        ],
        "childrenRight": [
            2,
            -1,
            10,
            7,
            6,
            -1,
            -1,
            9,
            -1,
            -1,
            14,
            13,
            -1,
            -1,
            -1
        ],
        "classes": [
            [
                0.333333333333,
                0.333333333333,
                0.333333333333
            ],
            [
                0.333333333333,
                0.0,
                0.0
            ],
            [
                0.0,
                0.333333333333,
                0.333333333333
            ],
            [
                0.0,
                0.326666666667,
                0.033

File size:

In [7]:
%%bash

du -h data.json

4.0K	data.json


### Run classification in JavaScript:

In [8]:
%%bash

if hash node 2/dev/null; then
    python -m SimpleHTTPServer 8877 & serve_pid=$!
    node AdaBoostClassifier.js http://127.0.0.1:8877/data.json 1 2 3 4
    kill $serve_pid
fi

2


127.0.0.1 - - [22/Nov/2017 00:00:51] "GET /data.json HTTP/1.1" 200 -
